In [2]:
import requests as r
import pandas as pd 
import numpy as np 
from bs4 import BeautifulSoup

player_info = r.get("https://fantasy.premierleague.com/api/bootstrap-static/").json()

In [2]:
playersdf = pd.DataFrame(player_info['elements'])

In [3]:
playersdf.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next',
       'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam',
       'news', 'news_added', 'now_cost', 'photo', 'points_per_game',
       'second_name', 'selected_by_percent', 'special', 'squad_number',
       'status', 'team', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'starts', 'expected_goals', 'expected_assists',
       'expected_goal_involvements', 'expected_goals_con

In [4]:
playersdf = playersdf[['id','first_name','web_name','team','team_code','now_cost','photo','total_points','value_season','minutes','ict_index_rank',
'element_type','bps','influence', 'creativity', 'threat','goals_scored','expected_goals','assists','expected_assists','selected_rank','chance_of_playing_next_round','chance_of_playing_this_round']]
formation_map = {
    1:"Goalkeeper",
    2:'Defender',
    3:"Midfielder",
    4:"Attacker"
}
playersdf['position']= playersdf['element_type'].map(formation_map)
playersdf['chance_of_playing_next_round'] = playersdf['chance_of_playing_next_round'].fillna(100.0)
playersdf

,id,first_name,web_name,team,team_code,now_cost,photo,total_points,value_season,minutes,...,creativity,threat,goals_scored,expected_goals,assists,expected_assists,selected_rank,chance_of_playing_next_round,chance_of_playing_this_round,position
0,1,Folarin,Balogun,1,3,45,232223.jpg,0,0.0,0,...,0.0,0.0,0,0.00,0,0.00,196,100.0,100.0,Attacker
1,2,Cédric,Cédric,1,3,40,58822.jpg,0,0.0,0,...,0.0,0.0,0,0.00,0,0.00,252,100.0,NaN,Defender
2,3,Mohamed,M.Elneny,1,3,45,153256.jpg,0,0.0,0,...,0.0,0.0,0,0.00,0,0.00,430,0.0,0.0,Midfielder
3,4,Fábio,Fábio Vieira,1,3,55,438098.jpg,0,0.0,0,...,0.0,0.0,0,0.00,0,0.00,415,100.0,NaN,Midfielder
4,5,Gabriel,Gabriel,1,3,50,226597.jpg,1,0.2,4,...,0.0,0.0,0,0.00,0,0.00,7,100.0,NaN,Defender
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,572,Nélson,N.Semedo,20,39,45,200402.jpg,0,0.0,0,...,0.0,0.0,0,0.00,0,0.00,276,100.0,NaN,Defender
654,573,Toti António,Toti,20,39,45,510362.jpg,0,0.0,0,...,0.0,0.0,0,0.00,0,0.00,438,100.0,NaN,Defender
655,574,Boubacar,Boubacar Traore,20,39,45,476502.jpg,0,0.0,0,...,0.0,0.0,0,0.00,0,0.00,223,100.0,NaN,Midfielder
656,590,Matheus,Cunha,20,39,55,430871.jpg,0,0.0,0,...,0.0,0.0,0,0.00,0,0.00,308,75.0,75.0,Attacker


In [5]:
def get_player_data(player_id):
    data = r.get("https://fantasy.premierleague.com/api/element-summary/{}".format(player_id)).json()
    return data

In [6]:
def player_stats(player_id):

    data = get_player_data(player_id)
    fixture_data = pd.DataFrame(data['fixtures'])
    player_history = pd.DataFrame(data['history_past'])

    team_fixture_score_mean_10 = fixture_data.head(10)['difficulty'].mean()
    team_fixture_score_sum_10  = fixture_data.head(10)['difficulty'].sum()
    team_fixture_score_mean = fixture_data['difficulty'].mean()
    team_fixture_score_sum  = fixture_data['difficulty'].sum()

    if player_history.shape[0] >0:
        if "2022/23" in player_history.season_name.values:
            goals_scored_last_season = player_history[player_history.season_name == "2022/23"]['goals_scored'].iloc[0]
            goals_conceded_last_season = player_history[player_history.season_name == "2022/23"]['goals_conceded'].iloc[0]
            assists_last_season = player_history[player_history.season_name == "2022/23"]['assists'].iloc[0]
            minutes_last_season = player_history[player_history.season_name == "2022/23"]['minutes'].iloc[0]
            clean_sheets_last_season = player_history[player_history.season_name == "2022/23"]['clean_sheets'].iloc[0]
            total_points_last_season = player_history[player_history.season_name == "2022/23"]['total_points'].iloc[0]
            ict_index_last_season = player_history[player_history.season_name == "2022/23"]['ict_index'].iloc[0]
            total_points_last_season = player_history[player_history.season_name == "2022/23"]['total_points'].iloc[0]
            red_cards_last_season = player_history[player_history.season_name == "2022/23"]['clean_sheets'].iloc[0]
            yellow_cards_last_season = player_history[player_history.season_name == "2022/23"]['yellow_cards'].iloc[0]
            starts_last_season = player_history[player_history.season_name == "2022/23"]['starts'].iloc[0]
            saves_last_season = player_history[player_history.season_name == "2022/23"]['saves'].iloc[0]
            penalties_saved_last_season = player_history[player_history.season_name == "2022/23"]['penalties_saved'].iloc[0]
            penalties_missed_last_season = player_history[player_history.season_name == "2022/23"]['penalties_missed'].iloc[0]
        else:
            (goals_scored_last_season,goals_conceded_last_season,assists_last_season,minutes_last_season,clean_sheets_last_season,
            clean_sheets_last_season,total_points_last_season,ict_index_last_season,red_cards_last_season,yellow_cards_last_season,
            starts_last_season,saves_last_season,penalties_saved_last_season,penalties_missed_last_season) = [0]*14
    else:
            (goals_scored_last_season,goals_conceded_last_season,assists_last_season,minutes_last_season,clean_sheets_last_season,
            clean_sheets_last_season,total_points_last_season,ict_index_last_season,red_cards_last_season,yellow_cards_last_season,
            starts_last_season,saves_last_season,penalties_saved_last_season,penalties_missed_last_season) = [0]*14


    
    return (team_fixture_score_mean_10,team_fixture_score_sum_10,team_fixture_score_mean,team_fixture_score_sum,
    goals_scored_last_season,goals_conceded_last_season,assists_last_season,minutes_last_season,clean_sheets_last_season,
    clean_sheets_last_season,total_points_last_season,ict_index_last_season,red_cards_last_season,yellow_cards_last_season,
    starts_last_season,saves_last_season,penalties_saved_last_season,penalties_missed_last_season)


In [7]:
player_stats(1)

(2.8, 28, 2.7567567567567566, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

In [8]:
data = np.transpose(np.array(list(playersdf.apply(lambda x: player_stats(x['id']),axis = 1 ))))
var_list = ['team_fixture_score_mean_10','team_fixture_score_sum_10','team_fixture_score_mean','team_fixture_score_sum',
'goals_scored_last_season','goals_conceded_last_season','assists_last_season','minutes_last_season','clean_sheets_last_season',
'clean_sheets_last_season','total_points_last_season','ict_index_last_season','red_cards_last_season','yellow_cards_last_season',
'starts_last_season','saves_last_season','penalties_saved_last_season','penalties_missed_last_season']
index = 0 
for x in var_list:
    playersdf[x] = data[index]
    index = index + 1


In [9]:
import pandas as pd
from pulp import LpProblem, LpVariable, lpSum, LpMaximize

data = playersdf

data['chance_of_playing_next_round'] = data['chance_of_playing_next_round'].astype(int)
data['total_points_last_season'] = data['total_points_last_season'].astype(int)
data['ict_index_last_season'] = data['ict_index_last_season'].astype(float)
data['team_fixture_score_sum_10'] = data['team_fixture_score_sum_10'].astype(float)

data = data[data.chance_of_playing_next_round == 100]
#removing Raya as he is transfering
data = data[data.id != 113]
data = data.reset_index(drop=True)

# Set budget constraint and position constraints
budget = 1000  # Set your budget limit
positions = ['Goalkeeper', 'Defender', 'Midfielder', 'Attacker']
position_counts = {'Goalkeeper': 2, 'Defender': 5, 'Midfielder': 5, 'Attacker': 3}
max_players_per_team = 3

# Create a PuLP linear programming problem
prob = LpProblem("FantasyFootball", LpMaximize)

# Create LpVariables for player selection
selected = [LpVariable(f"player_{i}", cat='Binary') for i in range(len(data))]

# Define the objective function (maximize total points)
prob += lpSum(selected[i] * data.loc[i, 'total_points_last_season'] for i in range(len(data))) + \
        lpSum(selected[i] * data.loc[i, 'ict_index_last_season'] for i in data.index) - \
        lpSum(selected[i] * data.loc[i, 'team_fixture_score_sum_10'] for i in data.index)

# Add budget constraint
prob += lpSum(selected[i] * data.loc[i, 'now_cost'] for i in range(len(data))) <= budget

# Add position constraints
for position in positions:
    prob += lpSum(selected[i] for i in range(len(data)) if data.loc[i, 'position'] == position) == position_counts[position]

# Add constraint for maximum players from the same team
team_players = {team: lpSum(selected[i] for i in data.index if data.loc[i, 'team'] == team)
                for team in data['team'].unique()}
for team, players in team_players.items():
    prob += players <= max_players_per_team

# Add specific player selections
# Ids for Archer,Holgate Lamptey, Ederson,Haaland and Forster
specific_player_ids  = [33,139,255,352,355,498]  # List of player ids I want to be included 
# Add specific player selections based on IDs
for i in data.index:
    if data.loc[i, 'id'] in specific_player_ids:
        prob += selected[i] == 1

# Solve the problem
prob.solve()

# Get the selected players
selected_players = [data.loc[i] for i in range(len(data)) if selected[i].varValue == 1]


pd.DataFrame(selected_players)

,id,first_name,web_name,team,team_code,now_cost,photo,total_points,value_season,minutes,...,minutes_last_season,clean_sheets_last_season,total_points_last_season,ict_index_last_season,red_cards_last_season,yellow_cards_last_season,starts_last_season,saves_last_season,penalties_saved_last_season,penalties_missed_last_season
9,12,Gabriel,Martinelli,1,3,80,444145.jpg,5,0.6,85,...,2789,14,198,298.1,14,3,34,0,0,0
11,14,Martin,Ødegaard,1,3,85,184029.jpg,2,0.2,90,...,3132,13,212,304.9,13,4,37,0,0,0
16,19,Bukayo,Saka,1,3,85,223340.jpg,10,1.2,90,...,3183,12,202,333.4,12,6,37,0,0,1
29,33,Cameron,Archer,2,7,45,433979.jpg,0,0.0,0,...,43,0,6,0.9,0,0,0,0,0,0
46,60,Ollie,Watkins,2,7,80,178301.jpg,5,0.6,90,...,3129,13,175,256.7,13,4,36,0,0,1
89,112,Ethan,Pinnock,4,94,45,231065.jpg,1,0.2,90,...,2700,10,112,127.2,10,0,30,0,0,0
109,134,Pascal,Gross,5,36,65,60307.jpg,2,0.3,90,...,3240,11,159,273.2,11,7,37,0,0,0
111,139,Tariq,Lamptey,5,36,40,232792.jpg,0,0.0,0,...,439,1,23,20.6,1,1,3,0,0,0
213,255,Mason,Holgate,9,11,40,194164.jpg,0,0.0,0,...,506,2,16,21.5,2,1,5,0,0,0
221,265,James,Tarkowski,9,11,45,17761.jpg,2,0.4,90,...,3420,9,106,167.0,9,6,38,0,0,0


In [4]:
fixtures = pd.DataFrame(r.get('https://fantasy.premierleague.com/api/fixtures').json())
fixtures[fixtures.id == 2]

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id
3,2367540,1.0,True,True,2,2023-08-12T12:00:00Z,90,False,True,16,1.0,1,2.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,4,93322


In [5]:
response = r.get("https://www.premierleague.com/match/"+str(fixtures[fixtures.id == 2]['pulse_id'].iloc[0]))
soup = BeautifulSoup(response.text, "html.parser")
stats = pd.DataFrame([[element.text for element in soup.find_all(class_="count")]],columns=['h_total_wins','h_home_wins','h_away_wins','a_total_wins','a_home_wins','a_away_wins'])
stats['h_win_perc'] = int(stats['h_total_wins']) / (int(stats['h_total_wins']) + int(stats['a_total_wins']))
stats['a_win_perc'] = int(stats['a_total_wins']) / (int(stats['h_total_wins']) + int(stats['a_total_wins']))
stats

,h_total_wins,h_home_wins,h_away_wins,a_total_wins,a_home_wins,a_away_wins,h_win_perc,a_win_perc
0,10,6,4,4,4,0,0.714286,0.285714
